In [1]:
import math
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.utils.np_utils import to_categorical


/home/daneel/anaconda3/envs/py36/lib/python3.6/site-packages/matplotlib/__init__.py:1038: UserWarning: Bad val "gtkAgg" on line #1
	"backend: gtkAgg
"
	in file "/home/daneel/.config/matplotlib/matplotlibrc"
	Key backend: Unrecognized backend string 'gtkagg': valid strings are ['GTK3Agg', 'GTK3Cairo', 'MacOSX', 'nbAgg', 'Qt4Agg', 'Qt4Cairo', 'Qt5Agg', 'Qt5Cairo', 'TkAgg', 'TkCairo', 'WebAgg', 'WX', 'WXAgg', 'WXCairo', 'agg', 'cairo', 'pdf', 'pgf', 'ps', 'svg', 'template']
  (val, error_details, msg))


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# Load the data
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train_X = train.iloc[:,1:]
train_y = train['label']

print(train_X.shape)
print(train_y.shape)
print(test.shape)

del train 

In [ ]:
train_y.value_counts()

In [ ]:
train_X.isnull().sum().sum()

In [ ]:
test.isnull().any().describe()

In [ ]:
# Normalize the data
train_X = train_X / 255.0
test = test / 255.0

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
train_X = train_X.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
train_y = to_categorical(train_y, num_classes = 10)

In [ ]:
# Split the train and the validation set for the fitting
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size = 0.33)


In [ ]:
plt.imshow(train_X[0][:,:,0])

In [ ]:

model = Sequential()


model.add(Conv2D(filters = 32, kernel_size = (2,2),
                 padding = 'Same', 
                 activation ='relu', 
                 input_shape = (28,28,1)))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(10, activation = "softmax"))

opt = Adam(lr=0.012, decay=0.001) # the decay param decreases the learning rate of that amount at each epoch

model.compile(optimizer = opt , loss = "categorical_crossentropy", metrics=["categorical_accuracy"])

In [ ]:
epochs = 15

datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1
)

datagen.fit(train_X)

# fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(train_X, 
                                 train_y, 
                                 batch_size=64),
                    validation_data = (val_X, val_y),
                    epochs=epochs)


In [ ]:
pred_y = model.predict(val_X)
print('RMSE = %.6f' % math.sqrt(mean_squared_error(pred_y, val_y)))


In [ ]:
sns.lineplot(x=range(len(history.history['val_categorical_accuracy'])), y=history.history['val_categorical_accuracy'], label='val_categorical_accuracy')
sns.lineplot(x=range(len(history.history['categorical_accuracy'])), y=history.history['categorical_accuracy'], label='categorical_accuracy')


In [ ]:
matrix = confusion_matrix(val_y.argmax(axis=1), pred_y.argmax(axis=1))

sns.heatmap(matrix, cbar=False, annot=True, fmt='d', cmap="Blues", square=True)


## Submission

In [ ]:
test_y = model.predict(test)
test_y = test_y.round()
out_label = [ np.argmax(i) for i in test_y]
out_imageid = [ i+1 for i in range(len(test_y))]
out = pd.DataFrame()
out['ImageId'] = out_imageid
out['Label'] = out_label

out.to_csv('submission.csv', index=False)
out